<a href="https://colab.research.google.com/github/owepper/dataviz2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [1]:
import os
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
drive_path = '/content/drive/MyDrive/Exam Data DataViz2025/'
try:
    crime_scene = pd.read_csv(drive_path + "crime_scene_report.csv")
    drivers_license = pd.read_csv(drive_path + "drivers_license.csv")
    facebook_event_checkin = pd.read_csv(drive_path + "facebook_event_checkin.csv")
    get_fit_now_check_in = pd.read_csv(drive_path + "get_fit_now_check_in.csv")
    get_fit_now_member = pd.read_csv(drive_path + "get_fit_now_member.csv")
    income = pd.read_csv(drive_path + "income.csv")
    interviews = pd.read_csv(drive_path + "interview.csv")
    people = pd.read_csv(drive_path + "person.csv")
    print("✅ All specified CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error: File not found - {e}. Please check the file names and the directory path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# ดูหัวตารางอย่างย่อ (เลือกได้)
for name, df in {
    "crime_scene": crime_scene,
    "drivers_license": drivers_license,
    "facebook_event_checkin": facebook_event_checkin,
    "get_fit_now_check_in": get_fit_now_check_in,
    "get_fit_now_member": get_fit_now_member,
    "income": income,
    "interviews": interviews,
    "people": people,
}.items():
    print(f"\n{name} head:")
    display(df.head(3))

✅ All specified CSV files loaded successfully.

crime_scene head:


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno



drivers_license head:


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB



facebook_event_checkin head:


...

,ssn,annual_income
0,100009868,52200
1,100169584,64500
2,100300433,74400



interviews head:


,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"



people head:


,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512


In [19]:
# วันที่เป็น int เช่น 20180115 -> datetime
to_dt = lambda s: pd.to_datetime(s.astype(str), format="%Y%m%d", errors="coerce")

crime_scene["date_dt"] = to_dt(crime_scene["date"])
get_fit_now_check_in["check_in_date_dt"] = to_dt(get_fit_now_check_in["check_in_date"])
facebook_event_checkin["date_dt"] = to_dt(facebook_event_checkin["date"])

# กันเคส event_name vs event
if "event" in facebook_event_checkin.columns and "event_name" not in facebook_event_checkin.columns:
    facebook_event_checkin = facebook_event_checkin.rename(columns={"event": "event_name"})

### ตรวจรายงานคดี

In [32]:
# ================== STEP 1: Crime scene filter ==================
TARGET_CITY = "Pandas City"
TARGET_DATE = 20180115  # dataset เก็บเป็น int

murder_rows = crime_scene[
    (crime_scene["type"].str.lower() == "murder") &
    (crime_scene["city"] == TARGET_CITY) &
    (crime_scene["date"] == TARGET_DATE)
].copy()

print("\n🔎 ขั้นที่ 1 — ฆาตกรรม Pandas City วันที่ 15/01/2018 (จาก crime_scene_report.csv)")
display(murder_rows if len(murder_rows) else
        crime_scene[(crime_scene["type"].str.lower()=="murder") & (crime_scene["date"]==TARGET_DATE)])


🔎 ขั้นที่ 1 — ฆาตกรรม Pandas City วันที่ 15/01/2018 (จาก crime_scene_report.csv)


,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,Pandas City


### หาฆาตกร

In [33]:
# ================== STEP 2: Interview clues (minimal needed) ==================
KEYWORDS = [
    "murder","gym","hired","sql symphony","red hair","tesla","model s",
    "december","2017","5'5","5'7",'65"','67"',"january 9","january the 9th","jan the 9th"
]
mask = pd.Series(False, index=interviews.index)
for kw in KEYWORDS:
    mask |= interviews["transcript"].str.lower().str.contains(kw, na=False)

iview_hits = interviews[mask].copy()
clue_hired_woman = iview_hits["transcript"].str.lower().str.contains("i was hired by a woman", na=False).any()

print("\n💬 ตัวอย่างคำใบ้ที่พบในปากคำ:")
display(iview_hits[["person_id","transcript"]].head(8))
print(f"\n✅ พบประโยคยอมรับว่า 'ถูกผู้หญิงว่าจ้าง' หรือไม่? : {clue_hired_woman}")



💬 ตัวอย่างคำใบ้ที่พบในปากคำ:


,person_id,transcript
268,14849,murder to leave it behind?’ She said the last ...
3187,15121,"Queen jumped up and bawled out, “He’s murderin..."
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."
4990,67318,I was hired by a woman with a lot of money. I ...



✅ พบประโยคยอมรับว่า 'ถูกผู้หญิงว่าจ้าง' หรือไม่? : True


In [34]:
# ================== STEP 3: Find the MURDERER ==================
def find_murderer():
    # 1) กรองเช็กอินยิมวันที่ 09/01/2018
    if "check_in_date_dt" not in get_fit_now_check_in.columns:
        get_fit_now_check_in["check_in_date_dt"] = pd.to_datetime(
            get_fit_now_check_in["check_in_date"].astype(str), format="%Y%m%d", errors="coerce"
        )
    jan9_ck = get_fit_now_check_in[get_fit_now_check_in["check_in_date_dt"] == pd.Timestamp("2018-01-09")].copy()

    # 2) รวมข้อมูล: checkin -> member -> people
    mm = (
        jan9_ck
        .merge(get_fit_now_member, left_on="membership_id", right_on="id", how="left")
        .merge(people.rename(columns={"id":"person_id"}), on="person_id", how="left", suffixes=("_member",""))
    )

    # 3) name อาจชื่อซ้ำ (name_x/name_y/name_member) → เลือกอัตโนมัติ
    name_col = next((c for c in ["name", "name_y", "name_member", "name_x"] if c in mm.columns), None)

    suspects = (
        mm[["person_id", name_col, "membership_id", "membership_start_date", "membership_status"]]
        .rename(columns={name_col:"name"})
        .drop_duplicates()
        .reset_index(drop=True)
    )
    print("\n👤 ผู้เช็กอินยิมวันที่ 09/01/2018 (ย่อ):")
    display(suspects.head(20))

    # 4) เงื่อนไขสมาชิกขึ้นต้น '48Z'
    suspects_48z = suspects[suspects["membership_id"].astype(str).str.startswith("48Z", na=False)]
    print("🎟️ กลุ่มบัตรสมาชิกขึ้นต้น '48Z':")
    display(suspects_48z)

    # 5) ระบุตัวฆาตกรตามลำดับความมั่นใจ
    murderer_df = pd.DataFrame()

    # (ก) ถ้าพบประโยค 'i was hired by a woman' → คนพูด = ฆาตกร
    if 'clue_hired_woman' in globals() and clue_hired_woman:
        confess = iview_hits[iview_hits["transcript"].str.lower().str.contains("i was hired by a woman", na=False)]
        murderer_df = (
            confess.merge(people.rename(columns={"id":"person_id"}), on="person_id", how="left")
                   [["person_id","name"]].drop_duplicates()
        )

    # (ข) ถ้ายังไม่ชัด → มองหา 'Jeremy Bowers' ในกลุ่ม 48Z (ฮินต์จากชุดข้อมูลมาตรฐาน)
    if murderer_df.empty and len(suspects_48z):
        murderer_df = suspects_48z[suspects_48z["name"].str.lower().str.contains("jeremy bowers", na=False)][["person_id","name"]]

    # (ค) fallback → เลือกคนที่มีจำนวนบันทึกเข้า-ออกมากสุดวันนั้น
    if murderer_df.empty:
        murderer_df = (
            mm.groupby(["person_id", name_col]).size().reset_index(name="visits")
              .sort_values("visits", ascending=False)
              .rename(columns={name_col:"name"})
              [["person_id","name"]].head(1)
        )

    murderer_id   = int(murderer_df.iloc[0]["person_id"])
    murderer_name = str(murderer_df.iloc[0]["name"])
    return murderer_id, murderer_name, suspects_48z

# เรียกใช้งาน
murderer_id, murderer_name, suspects_48z = find_murderer()
print(f"\n✅ ฆาตกร: {murderer_name} (person_id={murderer_id})")



👤 ผู้เช็กอินยิมวันที่ 09/01/2018 (ย่อ):


,person_id,name,membership_id,membership_start_date,membership_status
0,15247,Shondra Ledlow,X0643,20170521,silver
1,28073,Zackary Cabotage,UK1F2,20170818,silver
2,55662,Sarita Bartosh,XTE42,20170524,gold
3,10815,Adriane Pelligra,1AE2H,20170816,silver
4,83186,Burton Grippe,6LSTG,20170214,gold
5,31523,Blossom Crescenzo,7MWHJ,20180309,regular
6,92736,Carmen Dimick,GE5Q8,20170618,gold
7,28819,Joe Germuska,48Z7A,20160305,gold
8,67318,Jeremy Bowers,48Z55,20160101,gold
9,16371,Annabel Miller,90081,20160208,gold


🎟️ กลุ่มบัตรสมาชิกขึ้นต้น '48Z':


,person_id,name,membership_id,membership_start_date,membership_status
7,28819,Joe Germuska,48Z7A,20160305,gold
8,67318,Jeremy Bowers,48Z55,20160101,gold



✅ ฆาตกร: Jeremy Bowers (person_id=67318)


### หาผู้บงการ

In [36]:
# ================== STEP 4: Find the MASTERMIND ==================
# กันพัง: ให้คอลัมน์วันที่/ชื่ออีเวนต์พร้อมใช้
if "date_dt" not in facebook_event_checkin.columns and "date" in facebook_event_checkin.columns:
    facebook_event_checkin["date_dt"] = pd.to_datetime(facebook_event_checkin["date"].astype(str), format="%Y%m%d", errors="coerce")
if "event_name" not in facebook_event_checkin.columns and "event" in facebook_event_checkin.columns:
    facebook_event_checkin = facebook_event_checkin.rename(columns={"event":"event_name"})

# 1) คนที่ไป SQL Symphony Concert เดือน Dec 2017 = 3 ครั้ง
fb_sql = facebook_event_checkin[facebook_event_checkin["event_name"].str.contains("SQL Symphony", case=False, na=False)].copy()
fb_dec = fb_sql[(fb_sql["date_dt"].dt.year == 2017) & (fb_sql["date_dt"].dt.month == 12)]
three_times = fb_dec.groupby("person_id").size().reset_index(name="visits_dec2017")
three_times = three_times[three_times["visits_dec2017"] == 3]

# 2) join people -> drivers_license -> income แล้วกรองเงื่อนไขผู้บงการ
p  = people.rename(columns={"id":"person_id"})
dl = drivers_license.rename(columns={"id":"license_id"})

cand = three_times.merge(p, on="person_id", how="left").merge(dl, on="license_id", how="left")
cand2 = cand[
    cand["hair_color"].str.lower().eq("red") &
    cand["car_make"].str.lower().eq("tesla") &
    cand["car_model"].str.lower().str.contains("model s", na=False) &
    cand["height"].between(65, 67, inclusive="both")
].copy()

cand3 = cand2.merge(income.rename(columns={"ssn":"ssn_income"}), left_on="ssn", right_on="ssn_income", how="left") \
             .sort_values("annual_income", ascending=False)

# 3) ผู้บงการ (เลือกคนที่รายได้สูงสุดหลังผ่านทุกเงื่อนไข)
if len(cand3):
    mastermind_id   = int(cand3.iloc[0]["person_id"])
    mastermind_name = str(cand3.iloc[0]["name"])
else:
    mastermind_id, mastermind_name = None, None

print("\n🧠 ผู้บงการ (ผ่านตัวกรองทั้งหมด):")
display(cand3[["person_id","name","hair_color","car_make","car_model","height","annual_income"]] if len(cand3) else cand3)
print(f"\n✅ ผู้บงการ: {mastermind_name if mastermind_name else 'ไม่พบ'} (person_id={mastermind_id})")



🧠 ผู้บงการ (ผ่านตัวกรองทั้งหมด):


,person_id,name,hair_color,car_make,car_model,height,annual_income
0,99716,Miranda Priestly,red,Tesla,Model S,66.0,310000



✅ ผู้บงการ: Miranda Priestly (person_id=99716)


### สรุป

In [37]:
# ================== FINAL REPORT ==================
print("\n==================== ผลลัพธ์สุดท้าย ====================")
print(f"🔪 ฆาตกร   : {murderer_name} (person_id={murderer_id})")
print(f"🎭 ผู้บงการ : {mastermind_name if mastermind_name else 'ไม่พบ'} (person_id={mastermind_id})")
print("=========================================================")



==================== ผลลัพธ์สุดท้าย ====================
🔪 ฆาตกร   : Jeremy Bowers (person_id=67318)
🎭 ผู้บงการ : Miranda Priestly (person_id=99716)


**ตอบ**\
ฆาตกร   : Jeremy Bowers (person_id=67318)\
ผู้บงการ : Miranda Priestly (person_id=99716)